# OpenQAOA Example: Manual Mode

In [ ]:
from openqaoa.qaoa_parameters import PauliOp, Hamiltonian, QAOACircuitParams, create_qaoa_variational_params
from openqaoa.devices import DeviceQiskit, create_device
from openqaoa.backends.qaoa_backend import *
from openqaoa.optimizers import get_optimizer
from qiskit import QuantumCircuit
from openqaoa.utilities import *

### Define Cost and Mixer Hamiltonians with ease!

In [ ]:
cost_hamil = Hamiltonian([PauliOp('ZZ', (0, 1)), PauliOp('ZZ', (1, 2)), PauliOp('ZZ', (0, 2))], 
                         [1, 1, 1], 1)

In [ ]:
classical_terms = [(0, 1),(1, 2),(0, 2)]
coeffs = [1, 1, 1]
constant = 1
cost_hamil = Hamiltonian.classical_hamiltonian(terms=classical_terms,coeffs=coeffs,constant=constant)
cost_hamil.expression

In [ ]:
mixer_hamil = X_mixer_hamiltonian(n_qubits=3)

### Define QAOA Circuit Parameters
QAOA Circuit Params encode the cost and mixer hamiltonians in the form of an Abstract circuit which will later be used for constructing respective backend circuits

In [ ]:
circuit_params = QAOACircuitParams(cost_hamil, mixer_hamil, p=2)

### Define QAOA Variational Parameters
* Variational Params will be used to run QAOA computations to compute, for instance, expectation values. They are designed to be updated easily using an optimiser during VQA optimisation routines
* The user can specify the desired parameterization and initialization strategy 

In [ ]:
variate_params = create_qaoa_variational_params(circuit_params,'standard','ramp')

### Define the Backend of your choice!
* Choose from the available local/cloud based devices including IBMQ, Rigetti QPUs
* Depending on whether the chosen device is cloud-based or local, you may have to define an authentication dictionary with your credentials as follows

In [ ]:
auth_dict = { "QISKIT": 
             { "API_TOKEN": "<API TOKEN>",
              "HUB": "ibm-q", 
              "GROUP": "open", 
              "PROJECT": "main"}
            }

selected_device = 'ibmq_qasm_simulator'

device_qiskit = DeviceQiskit(auth_dict["QISKIT"]["API_TOKEN"], auth_dict["QISKIT"]["HUB"], 
                   auth_dict["QISKIT"]["GROUP"], auth_dict["QISKIT"]["PROJECT"], selected_device)

In [ ]:
backend_cloud = get_qaoa_backend(circuit_params, device_qiskit, n_shots=1000)

In [ ]:
backend_cloud.parametric_circuit.draw()

In [ ]:
backend_cloud.qaoa_circuit(variate_params).draw()

### Creating other backends

In [ ]:
qiskit_sim = create_device('local','qiskit.statevector_simulator')
backend_qiskit_statevec = get_qaoa_backend(circuit_params, qiskit_sim)

In [ ]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ
# IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q-startup','entropica-labs','reservations')
backend = provider.get_backend('ibmq_bogota')
noise_model = NoiseModel.from_backend(backend)

In [ ]:
qiskit_shot_sim = create_device('local','qiskit.shot_simulator')
backend_qiskit_shot = get_qaoa_backend(circuit_params, qiskit_shot_sim, n_shots=100, noise_model=noise_model)

In [ ]:
print(backend_qiskit_statevec.expectation(variate_params))
print(backend_qiskit_statevec.expectation_w_uncertainty(variate_params))

In [ ]:
print(backend_qiskit_shot.expectation(variate_params))
print(backend_qiskit_shot.expectation_w_uncertainty(variate_params))

In [ ]:
current_ckt = backend_qiskit_shot.qaoa_circuit(variate_params)

In [ ]:
#print parametric qaoa circuit
backend_qiskit_shot.parametric_circuit.draw()

In [ ]:
#print the full circuit with attached angles
current_ckt.draw()

In [ ]:
backend_cloud.expectation_w_uncertainty(variate_params)

In [ ]:
backend_qiskit_statevec.qfim(variate_params)

In [ ]:
optimizer_dict = {'method': 'cobyla', 'maxiter': 10}

In [ ]:
optimizer_obj = get_optimizer(backend_qiskit_shot, variate_params, optimizer_dict)

In [ ]:
optimizer_obj

In [ ]:
optimizer_obj()
optimizer_obj.results_information()

In [ ]:
optimizer_obj()
optimizer_obj.results_information()

### Some useful functions to play around with!

In [ ]:
from openqaoa.utilities import plot_energy_spectrum

In [ ]:
plot_energy_spectrum(cost_hamil)